In [1]:
from lmfit import minimize
from lmfit import Parameters

In [2]:
import pyroll.core as pr
import pyroll.pillar_model
import pyroll.neutral_point_estimator
import pyroll.sparling_spreading
import pyroll.hensel_force_torque
import pyroll.freiberg_flow_stress
import pyroll.ring_model_thermal

In [3]:
from in_profile import create_in_profile_from_surface_temperature
from rolling_sequence import roughing_sequence, intermediate_sequence, finishing_sequence_e112, finishing_sequence_e107
from utilities import *

In [4]:
pyroll.pillar_model.Config.PILLAR_COUNT = 30
pyroll.neutral_point_estimator.Config.ESTIMATOR = "EQUAL"
ROLL_SURFACE_TEMPERATURE = 50 + 273.15
DISK_ELEMENT_COUNT = 30

In [5]:
@pr.CoolingPipe.DiskElement.heat_transfer_coefficient
def heat_transfer_coefficient_model_moreales(self: pr.CoolingPipe.DiskElement):
    mean_surface_temperature = (self.in_profile.surface_temperature + self.out_profile.surface_temperature) / 2
    profile_surface_temperature_celsius = mean_surface_temperature - 273.15
    return 13950 + 5.12 * 1e6 / profile_surface_temperature_celsius

In [6]:
in_profile = create_in_profile_from_surface_temperature(1050 + 273.15)
in_profile.ring_temperatures = in_profile.ring_temperatures

In [7]:
roughing_train = roughing_sequence(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)

intermediate_train = intermediate_sequence(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)

finishing_train_e112 = finishing_sequence_e112(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)

finishing_train_e107 = finishing_sequence_e107(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)


In [8]:
rolling_train_e112 = pr.PassSequence([
    roughing_train,
    intermediate_train,
    finishing_train_e112
])

rolling_train_e107 = pr.PassSequence([
    roughing_train,
    intermediate_train,
    finishing_train_e107
])

In [9]:
rolling_train_e112.flatten()
rolling_train_e107.flatten()

In [10]:
rolling_train_e112.solve(in_profile)

Solution iteration of TwoRollPass '3' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '7' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '14' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '18-V' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '23-H' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '18-V' exceeded the maximum iteration count of 100. Continuing anyway.


Profile(
    classifiers={'false_round', 'generic_elongation', 'round', 'symmetric'},
    core_temperature=np.float64(1570.0591539343975),
    cross_section=Polygon(
        area=5.939031536528555e-05,
        height=0.008700000000000006,
        perimeter=0.02736933246437044,
        width=0.008768227336460354,
        ),
    length=5171.222724142025,
    material=['BST500', 'steel'],
    ring_temperatures=array([1570.05915393, 1567.12701904, 1558.34092613, 1543.71864333,
           1523.23735286, 1496.84675482, 1464.74408402, 1428.15944014,
           1390.54359123, 1358.13149   , 1337.87659106]),
    strain=np.float64(0.5607017311212023),
    surface_temperature=np.float64(1337.8765910557204),
    t=np.float64(36.875179665279155),
    temperature=np.float64(1425.8613020468922),
    weight=2400,
    width=np.float64(0.008768227336460354),
    )

In [11]:
rolling_train_e107.solve(in_profile)

Solution iteration of TwoRollPass '22-V' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '23-H' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '27-H' exceeded the maximum iteration count of 100. Continuing anyway.


Profile(
    classifiers={'false_round', 'generic_elongation', 'round', 'symmetric'},
    core_temperature=np.float64(1564.024619885944),
    cross_section=Polygon(
        area=5.896220122998807e-05,
        height=0.008700000000000006,
        perimeter=0.027243885509781032,
        width=0.008532317323115969,
        ),
    length=5208.7339449078745,
    material=['BST500', 'steel'],
    ring_temperatures=array([1564.02461989, 1561.13466478, 1552.47023573, 1538.03228003,
           1517.78168798, 1491.70612299, 1460.17725746, 1424.73026868,
           1388.99526951, 1358.82742219, 1340.17168701]),
    strain=np.float64(0.520194900008893),
    surface_temperature=np.float64(1340.1716870065236),
    t=np.float64(36.854174411047744),
    temperature=np.float64(1423.7200086555574),
    weight=2400,
    width=np.float64(0.008532317323115969),
    )

In [12]:
def extract_wear_contours_from_measurement(groove_name):
    root_dir = Path.cwd()
    wear_data_dir = root_dir / "wear_data" / groove_name

    labels = []
    tonnages = []
    measured_wear_contours = []

    for file in wear_data_dir.iterdir():
        with open(file, "r") as file:
            data = json.load(file)

        for entry in data["wear_data"]:
            labels.append(entry["stand"])
            tonnages.append(entry["tonnage"])
            x_values = np.array([point["x"] * 1e-3 for point in entry["wear_contour"]])
            y_values = np.array([point["y"] * 1e-3 for point in entry["wear_contour"]])
            x_values_shifted = x_values - max(x_values) / 2
            measured_wear_contour = LineString(list(zip(x_values_shifted, y_values)))
            measured_wear_contours.append(measured_wear_contour)

        return labels, tonnages, measured_wear_contours

In [13]:
def solve_total_wear_depths_as_contour_line(params):
    sequences = [
        {
            "label": "e11_2",
            "sequence": rolling_train_e112
        },
        {
            "label": "e10_7",
            "sequence": rolling_train_e107
        }
    ]

    area_errors = []
    wear_coefficient = params['wear_coefficient'].value

    for dict in sequences:
        roll_passes = dict["sequence"].roll_passes
        for roll_pass in roll_passes:
            labels, tonnages, measured_wear_contours = extract_wear_contours_from_measurement(dict["label"])
            for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                if roll_pass.label == label:
                    calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                    area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)

                    area_errors.append(area_error)
                else:
                    continue
    print(area_errors)
    rms_error = root_mean_square_value(np.array(area_errors))
    print("RMS Error: ", rms_error)
    return rms_error


In [14]:
params = Parameters()
params.add('wear_coefficient', value=1e-6, min=1e-10, max=1e-3, vary=True)

In [15]:
result = minimize(
    solve_total_wear_depths_as_contour_line,
    params,
    nan_policy='omit',
    max_nfev=100,
    method="least_squares"
    #method="cobyla"
)

[np.float64(0.016860056411351323), np.float64(0.010242162329654846), np.float64(0.047391692095124935), np.float64(0.010852149667409258), np.float64(0.02330440026675892), np.float64(0.009021438173621835), np.float64(0.017006147722649), np.float64(0.008283567594700399), np.float64(0.00916502281013381), np.float64(0.005325686148000485), np.float64(0.010864787856011167), np.float64(0.009815914370343614), np.float64(0.006643524186903544), np.float64(0.008032536829695711), np.float64(0.00491491035087566), np.float64(0.004977521965445099)]
RMS Error:  0.01623953411191651
[np.float64(0.017111312595946594), np.float64(0.010394796296688902), np.float64(0.048097977817959485), np.float64(0.011013862098912747), np.float64(0.023651705284711447), np.float64(0.009155885360842374), np.float64(0.01725960483014659), np.float64(0.008407010079350227), np.float64(0.009302451656818358), np.float64(0.005405202917817698), np.float64(0.011027271961145713), np.float64(0.00996234360663308), np.float64(0.006742882

In [16]:
result.params

name,value,standard error,relative error,initial value,min,max,vary
wear_coefficient,4.1328e-09,4.0045e-09,(96.89%),1e-06,1.0000e-10,1.0000e-03,True


In [17]:
result

In [18]:
result.fit_report()

AttributeError: 'MinimizerResult' object has no attribute 'fit_report'

In [19]:
for roll_pass in rolling_train_e107.roll_passes:
    print(roll_pass.label)
    print(roll_pass.elongation)

A
1.332721444396918
B
1.2676741917624201
1
1.4148755445109202
2
1.2721075360738472
3
1.3431342336241443
4
1.295319732453617
5
1.2694938287140918
6
1.2086014744540545
7
1.300440193519346
8
1.2055568625036193
9
1.3703432274493266
10
1.2652280845613628
11
1.3547292905376056
12
1.2209624253366764
13
1.329132080637813
14
1.1957896857909633
17-H
1.3163849710650921
18-V
1.1799461031839393
21-H
1.3748433264386175
22-V
1.2019025294343542
23-H
1.3529498625258818
24-V
1.296311857760552
27-H
1.300416795126196
28-V
1.2783694269760837
